[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/plot_monitoring_data.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/plot_monitoring_data.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Fplot_monitoring_data.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] |
|:---------------------:|:---------------:|:-------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] |

## Introduction
This notebook provides an example of using the public REST API of WebAccess/DMP.

In particular, we will be using the `GET /monitoring/devices/query` endpoint, which you can also try out on https://api.wadmp.com/

![endpoint](images/GET_monitoring_devices_query.png)

Monitoring data is stored in a time-series database called [InfluxDB](https://www.influxdata.com/products/influxdb-overview/).

We use the [bqplot](https://github.com/bloomberg/bqplot) plotting library,
but feel free to adapt the notebook to use whatever tools you prefer.

### Requirements
* If you are running in Jupyter Notebook, you don't need to make any code changes.
* If you are running in Jupyter Lab, you will also need to install these JupyterLab extensions:
  * @jupyter-widgets/jupyterlab-manager
  * bqplot
* You need to have an existing user account on the WA/DMP instance.
  * You must be a member of at least one company on the WA/DMP instance;
  * At least one device that is claimed to said company must be configured to publish monitoring data.

### Usage
In the "Global Variables" cell below, change BASE_URL to match the particular WA/DMP instance that you are using.

Then run the cells, either one at a time, or all at once.

When prompted, enter the required User Input (USERNAME, PASSWORD, device_name).

## Setup
This may take a minute ...

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyjwt

# We have had issues with bqplot due to version incompatibilities.
# To be safe, we pin both the backend (pip) version and the frontend (npm) version.
!{sys.executable} -m pip install bqplot==0.12.12
# bqplot includes ipywidgets, numpy, pandas, so we don't need to install those.

# The following line is commented out because it will cause an exception in Jupyter Notebook.
# HOWEVER, it may be required in Jupyter Lab if you can't install the right version through the Extension Manager.
#!jupyter labextension install bqplot@0.5.12

import requests
import jwt
import numpy as np
import pandas as pd
import bqplot
import json
import datetime

## Global variables

In [ ]:
BASE_URL = 'https://gateway.wadmp.com'
BASE_PATH = 'api'
SESSION = requests.Session()  # Use one HTTPS session for all API calls

## Functions to be used later

In [ ]:
def login(username, password):
    """Login to the system, and return a token
    """
    url = f"{BASE_URL}/public/auth/connect/token"
    credentials = {'username': username, 'password': password, 'client_id': 'python', 'grant_type': 'password'}
    print(f"Sending POST request to {url} with:\n"
          f"    credentials={credentials}\n")
    response = SESSION.post(url, data=credentials)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()["access_token"]
    else:
        print("Failed to login!")
        sys.exit(1)


def get_companies(name=None):
    """Gets the list of companies in the system.
    """
    url = f"{BASE_URL}/{BASE_PATH}/companies"
    query = {'name': name}
    print(f"Sending GET request to {url} with:\n"
          f"    name={name}\n")
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of companies!")
        return None


def get_devices(page_size, companies=[]):
    """Retrieves the list of your devices.
       Requests are paged, but this function automatically aggregates responses into one complete list.
    """
    page_number = 1
    total, devices = get_one_page_of_devices(page_number, page_size, companies)
    
    while len(devices) < total:
        print(f"{len(devices)} out of {total} ...")
        page_number += 1
        total, page = get_one_page_of_devices(page_number, page_size, companies)
        devices.extend(page)

    return devices


def get_one_page_of_devices(page_number, page_size, companies=[]):
    """Retrieves one page of the list of your devices.
    """
    url = f"{BASE_URL}/{BASE_PATH}/management/devices"

    # The only REQUIRED query parameters are page and pageSize
    print(f"Sending GET request to {url} with:\n"
          f"    page={page_number}\n"
          f"    pageSize={page_size}")
    query = {'page': page_number, 'pageSize': page_size, 'companies': companies}
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)
    
    total = response.json()['total_items']

    if response.status_code == requests.codes['ok']:
        return total, response.json()['data']
    else:
        print(f"Failed to retrieve page {page_number}!")
        return None


def get_monitoring_data(influx_query):
    """Queries for monitoring data from a specific device
    """
    url = f"{BASE_URL}/{BASE_PATH}/monitoring/devices/query"
    query = {'Q': influx_query, 'Epoch': 'ms'}
    print(f"\nSending GET request to {url} with:\n"
            f"    Q={influx_query}\n"
            f"    Epoch=ms\n")
    response = SESSION.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:       
        if response.json()['results'][0]['series']:
            return response.json()['results'][0]['series']
        else:
            print("No data")
            return None
    else:
        print("InfluxDB query failed!")
        return None

## User input

In [ ]:
USERNAME = input("Enter WebAccess/DMP username:")
PASSWORD = input("Enter password:")

## Login to server

In [ ]:
user_token = login(USERNAME, PASSWORD)
SESSION.headers.update({'Authorization': f'Bearer {user_token}'})

## Select a Company
WebAccess/DMP utilises a separate InfluxDB database for every tenant, or "company".

In [ ]:
companies = get_companies()

In [ ]:
# A nested list comprehension creates a 2D array.
data = [[
    company['name'],
    company['id'],
    company['contact_name']
    ] for company in companies]

# Convert the 2D array into a Pandas dataframe, because it renders nicely
company_table = pd.DataFrame(data, columns=["name", "id", "contact_name"])
company_table.style.hide_index()

### Now, using the "id" column of this table ...

In [ ]:
company_id = input("Enter the ID of the company whose database you want to use:")

company_name = company_table.query(f"id=={company_id}")['name'].values[0]
print(f"Confirming company name = {company_name}")

## Select a device from this company

In [ ]:
devices = get_devices(100, [company_id])

In [ ]:
# A nested list comprehension creates a 2D array.
# We can't include "Parent Name" information yet, because a company may not have a parent
data = [[
    device['display_name'],
    device['serial_number'],
    device['mac_address'],
    device['device_type']['name'],
    device['is_online']
    ] for device in devices]

# Convert the 2D array into a Pandas dataframe, because it renders nicely
device_table = pd.DataFrame(data, columns=["Name", "Serial Number", "MAC Address", "Type", "Online?"])
device_table.style.hide_index()

### Now, using the "MAC Address" column of this table ...

In [ ]:
device_mac = input("Enter the MAC Address of the device:")

## Select a field from the database for this device

### Notes on creating an Influx query:
* The company name must be included in the WHERE clause!
(If you do not specify a company, the query will default to use the InfluxDB database for your primary company).

* Because of the way Grafana [uses variables in queries](https://grafana.com/docs/grafana/latest/features/datasources/influxdb/#using-variables-in-queries), you have to wrap the company name as follows:
`"companyName" =~ /^My Company Inc.$/`

* In our InfluxDB schema, we have some keys that are used for both a field and a tag. "macAddress" is one of these, so when using it in a query, we should specify the type.

* Following the Influx recommendations [here](https://docs.influxdata.com/influxdb/v1.7/troubleshooting/frequently-asked-questions/#when-should-i-single-quote-and-when-should-i-double-quote-in-queries), we single-quote string values and double-quote identifiers.

In [ ]:
influx_query = f'SELECT last(*) FROM "SNMP" WHERE ("macAddress"::tag = \'{device_mac}\' AND "companyName" =~ /^{company_name}$/)'
dictionary = get_monitoring_data(influx_query)[0]

Then we get the latest value and timestamp for each field ...

In [ ]:
fields = [field.split('_', maxsplit=1)[1] for field in dictionary['columns'] if field != 'time']
# Create a 2D array
array = []
for field in fields:
    influx_query = f'SELECT last({field}), time FROM "SNMP" WHERE ("macAddress"::tag = \'{device_mac}\' AND "companyName" =~ /^{company_name}$/)'
    results = get_monitoring_data(influx_query)
    if results:  # may be null
        values = results[0]['values'][0]
        timestamp = values[0]/1000.0
        value = values[1]
        array.append([field, datetime.datetime.fromtimestamp(timestamp, datetime.timezone.utc).strftime('%Y-%m-%d %H:%M:%S'), value])

In [ ]:
# Convert the 2D array into a Pandas dataframe, because it renders nicely
field_table = pd.DataFrame(array, columns=["Field", "Timestamp", "Value"])
field_table.style.hide_index()

### Now, using the "Field" column of this table ...
Note that not all fields have numeric values. You can query *any* field, but if you want to plot it, it is best to select a numeric field.

In [ ]:
field_name = input("Enter the name of the field you want to plot:")

## Get the last 24 hours of data

In [ ]:
influx_query = f'SELECT "{field_name}" FROM "SNMP" WHERE ("macAddress" = \'{device_mac}\' AND "companyName" =~ /^{company_name}$/ AND time >= now() - 1d )'
# Convert to a numpy array, because that makes slicing easy
array = np.array(get_monitoring_data(influx_query)[0]['values'])

### Convert the numpy array to a Pandas Series object

In [ ]:
x = pd.to_datetime(array[:,0], unit='ms')
y = array[:,1]
series = pd.Series(data=y, index=x)
series

### Plot

In [ ]:
# bqplot provides a high-level API called pyplot, which is intended to be similar to matplotlib.
# We use the lower-level "object API", as it provides more flexibility. It is based on the "Grammar of Graphics".

x_sc = bqplot.DateScale()
y_sc = bqplot.LinearScale()

ax_x = bqplot.Axis(label='Time', scale=x_sc, grid_lines='solid')
ax_y = bqplot.Axis(label=field_name, scale=y_sc, orientation='vertical', grid_lines='solid')

line = bqplot.Lines(x=x, y=y, scales={'x': x_sc, 'y': y_sc}, marker='circle')

fig = bqplot.Figure(axes=[ax_x, ax_y], marks=[line], title=device_mac)
fig

Note that the X-axis automatically uses the browser's local timezone.
All monitoring data in WebAccess/DMP is stored in UTC, so if your local time is NOT UTC you will notice an offset.

## Go back to the "Select a field for plotting" cell and plot another field ...